In [3]:
import random

class Character:
    def __init__(self):
        self.faction = ""
        self.additional_choice = ""

    def choose_faction(self):
        factions = [
            "Architekten", "Borstenmütter", "Chronomaster", "Cybersamurai", "Corporat", 
            "Herren der verlorenen See", "Krieger von Valhalla", "Kinder des Löwen", 
            "Marduk-Orden", "Moon Circus", "Nomads", "Pan Doa", "Schwesternschaft der 7", 
            "Der Schwarm", "Union", "Wanderer"
        ]
        
        # Add the 'Randomize Faction' option as the last choice
        factions.append("Randomize Faction")

        print("Choose your faction:")
        for i, faction in enumerate(factions, 1):
            print(f"{i}. {faction}")
        
        while True:
            try:
                choice = int(input("Enter the number of your choice: "))
                if 1 <= choice <= len(factions):
                    if factions[choice - 1] == "Randomize Faction":
                        self.random_faction_selection(factions[:-1])  # Pass only actual factions
                    else:
                        self.faction = factions[choice - 1]
                        print(f"You chose the {self.faction}!\n")
                    break
                else:
                    print("Please choose a valid number from the list.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def random_faction_selection(self, factions):
        self.faction = random.choice(factions)
        print(f"Randomly selected faction: {self.faction}!\n")

    def faction_specific_steps(self):
        # Define different steps for each faction
        if self.faction == "Architekten":
            self.additional_choice = self.choose_material()
        elif self.faction == "Borstenmütter":
            self.additional_choice = self.choose_beast()
        elif self.faction == "Chronomaster":
            self.additional_choice = self.choose_time_artifact()
        # Add similar structures for all 16 factions
        else:
            print("More steps for other factions can be added here!")

    def choose_material(self):
        materials = ["Stone", "Steel", "Glass"]
        print("Choose a material to build with:")
        for i, material in enumerate(materials, 1):
            print(f"{i}. {material}")
        choice = int(input("Enter the number of your choice: "))
        return materials[choice - 1]

    def choose_beast(self):
        beasts = ["Boar", "Wolf", "Bear"]
        print("Choose your beast companion:")
        for i, beast in enumerate(beasts, 1):
            print(f"{i}. {beast}")
        choice = int(input("Enter the number of your choice: "))
        return beasts[choice - 1]

    def choose_time_artifact(self):
        artifacts = ["Sundial", "Hourglass", "Clockwork Mechanism"]
        print("Choose a time artifact to master:")
        for i, artifact in enumerate(artifacts, 1):
            print(f"{i}. {artifact}")
        choice = int(input("Enter the number of your choice: "))
        return artifacts[choice - 1]

# Run the character creator
def main():
    character = Character()
    character.choose_faction()
    character.faction_specific_steps()

    print("\nCharacter Summary:")
    print(f"Faction: {character.faction}")
    print(f"Special Choice: {character.additional_choice}")

if __name__ == "__main__":
    main()


Choose your faction:
1. Architekten
2. Borstenmütter
3. Chronomaster
4. Cybersamurai
5. Corporat
6. Herren der verlorenen See
7. Krieger von Valhalla
8. Kinder des Löwen
9. Marduk-Orden
10. Moon Circus
11. Nomads
12. Pan Doa
13. Schwesternschaft der 7
14. Der Schwarm
15. Union
16. Wanderer
17. Randomize Faction


Enter the number of your choice:  17


Randomly selected faction: Nomads!

More steps for other factions can be added here!

Character Summary:
Faction: Nomads
Special Choice: 
